## File paths and train,validation split

In [1]:
import pandas as pd
import tqdm
import numpy as np

labels_file = 'labels.csv'
ukb_field_to_icd10_map_file = 'icd10_codes_mod.tsv'
output_prefix = 'ukb_real'


## Read icd10 mapping file and defined index label link

In [2]:
icdict ={}
icdcodes = []
with open(ukb_field_to_icd10_map_file,'r') as f:
    for l in f:
        lvals=l.strip().split()
        icdict[lvals[0]]=lvals[5]
        icdcodes.append(lvals[5])

i = -1
label_dict = {}
with open(labels_file,'r') as f:
    for l in f:
        label_dict[l.strip().split(' ')[0]]=i
        i += 1

# hard coded sex and dob
icdict['f.31.0.0'] = "sex"
icdict['f.34.0.0'] = "YEAR"
icdict['f.52.0.0'] = "MONTH"
icdict['f.40000.0.0'] = "Death"

# cancer fields
for j in range(17):
    icdict['f.40005.'+str(j)+'.0'] = "cancer_date_"+str(j)
    icdict['f.40006.'+str(j)+'.0'] = "cancer_type_"+str(j)

# cancer hes fields 
#for j in range(213):
#    icdict['f.41270.0.'+str(j)] = "hicd_"+str(j)
#    icdict['f.41280.0.'+str(j)] = "hicd_date_"+str(j)

icdict['f.53.0.0'] = "assessment_date"
icdict['f.21001.0.0']="BMI"
icdict['f.1239.0.0']="smoking"
icdict['f.1558.0.0']="alcohol"

len_icd = len(icdcodes)
#icdcodes.extend(['Death','assessment_date']+['cancer_date_'+str(j) for j in range(17)]+['hicd_date_'+str(j) for j in range(213)])
icdcodes.extend(['Death','assessment_date']+['cancer_date_'+str(j) for j in range(17)])


## Read ukb basket file in chunks, select icd10 code occurance and dates, format for delphi

In [13]:
import polars as pl
import os
import tqdm

path = "/lustre/groups/shared/ukbb-87065/dataset/phenos"
out_path = "/lustre/groups/shared/ukbb-87065/users/lucas.arnoldt/"

eid = pl.read_csv(os.path.join(path, "eid.csv"))
eid_col = eid.columns[0]
eid_series = eid[eid_col]

def key_to_filename(key: str):
    _, a, b, c = key.split(".")
    return f"{a}-{b}.{c}.csv"

def rename_col(col: str):
    if col == "eid":
        return col
    a, rest = col.split("-")
    b, c = rest.split(".")
    return f"f.{a}.{b}.{c}"

phenos = []

for key in tqdm.tqdm(icdict.keys()):
    filename = key_to_filename(key)
    fullpath = os.path.join(path, filename)

    df = pl.read_csv(fullpath)

    pheno_col = df.columns[0]

    df = df.rename({pheno_col: rename_col(pheno_col)})

    phenos.append(df.lazy())

big = pl.concat(phenos, how="horizontal")

result = big.with_columns([
    eid_series.alias("eid")
]).select(["eid", *big.columns])

result.sink_csv(os.path.join(out_path, "merged_phenos.csv"))

100%|██████████| 1171/1171 [01:17<00:00, 15.02it/s]
/tmp/ipykernel_1384666/613892829.py:41: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  ]).select(["eid", *big.columns])


In [27]:
dd = pd.read_csv(os.path.join(out_path, "merged_phenos.csv"))
dd = dd.rename(columns={"eid": "f.eid"})
dd = dd.set_index("f.eid")

/tmp/ipykernel_1384666/4199806718.py:1: DtypeWarning: Columns (1,2,3,4,6,7,8,9,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,85,87,89,90,91,92,93,94,95,96,97,98,99,100,101,103,104,105,108,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,158,159,160,162,163,164,165,166,167,168,169,170,171,172,173,175,176,177,178,179,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,198,200,201,202,204,205,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,244,245,246,247,248,249,250,251,252,253,255,256,258,259,262,263,264,265,266,267,268,269,270,271,272,274,275,276,277,278,279,281,282,283,284,285,286,2

In [29]:
data_list = []

dd = dd.rename(columns=icdict)
dd.dropna(subset=['sex'], inplace=True)
dd['sex'] += 1
dd = dd[[col for col in dd.columns if not col.startswith('f.')]]
dd['dob'] =  pd.to_datetime(dd[['YEAR', 'MONTH']].assign(DAY=1))
dd[icdcodes] = dd[icdcodes].apply(pd.to_datetime, format="%Y-%m-%d")
dd[icdcodes]=dd[icdcodes].sub(dd['dob'], axis=0)
dd[icdcodes]=dd[icdcodes].apply(lambda x : x.dt.days)

for col in icdcodes[:len_icd+1]:
    X = dd[col].dropna().reset_index().to_numpy().astype(int)
    data_list.append(np.hstack((X,label_dict[col]*np.ones([X.shape[0],1],X.dtype))))

X = dd['sex'].reset_index().to_numpy().astype(int)
data_list.append(np.c_[X[:,0],np.zeros(X.shape[0]),X[:,1]].astype(int))

for j in range(17):
    dd_cancer = dd[['cancer_date_'+str(j),'cancer_type_'+str(j)]].dropna().reset_index()
    if not dd_cancer.empty:
        dd_cancer['cancer'] = dd_cancer['cancer_type_'+str(j)].str.slice(0,3)
        dd_cancer['cancer_label'] = dd_cancer["cancer"].map(label_dict)
        data_list.append(dd_cancer[['f.eid','cancer_date_'+str(j),'cancer_label']].dropna().astype(int).to_numpy())

#for j in range(213):
#    dd_hicd = dd[['hicd_date_'+str(j),'hicd_'+str(j)]].dropna().reset_index()
#    if not dd_hicd.empty:
#        dd_hicd['hicd'] = dd_hicd['hicd_'+str(j)].str.slice(0,3)
#        dd_hicd['hicd_label'] = dd_hicd["hicd"].map(label_dict)
#        data_list.append(dd_hicd[['f.eid','hicd_date_'+str(j),'hicd_label']].dropna().astype(int).to_numpy())
    
dd_bmi = dd[['assessment_date','BMI']].dropna().reset_index()
dd_bmi['bmi_status'] = np.where(dd_bmi['BMI']>28,5,np.where(dd_bmi.BMI>22,4,3))
data_list.append(dd_bmi[['f.eid','assessment_date','bmi_status']].astype(int).to_numpy())

dd_sm = dd[['assessment_date','smoking']].dropna().reset_index()
dd_sm = dd_sm[dd_sm['smoking']!=-3]
dd_sm['smoking_status'] = np.where(dd_sm['smoking']==1,8,np.where(dd_sm.smoking==2,7,6))
data_list.append(dd_sm[['f.eid','assessment_date','smoking_status']].astype(int).to_numpy())

dd_al = dd[['assessment_date','alcohol']].dropna().reset_index()
dd_al = dd_al[dd_al['alcohol']!=-3]
dd_al['alcohol_status'] = np.where(dd_al['alcohol']==1,11,np.where(dd_al.alcohol < 4,10,9))
data_list.append(dd_al[['f.eid','assessment_date','alcohol_status']].astype(int).to_numpy())

## reformat, split train and val and output to delphi format

In [53]:
os.makedirs("../ukb_real_data/", exist_ok=True)

In [81]:
data = np.vstack(data_list)

data = data[np.lexsort((data[:,1], data[:,2]==data[:,2].max(), data[:,0]))]
data = data[data[:,1] >= 0]
data = pd.DataFrame(data).drop_duplicates([0,2]).values
data = data.astype(np.uint32)

data.tofile("../ukb_real_data/" + output_prefix + '.bin')

train_ids = pd.read_csv('/lustre/groups/shared/ukbb-87065/ukbb-found/data_benedikt/splits/train_ids_seed2024.csv')['id'].values.astype(np.uint32)
val_ids   = pd.read_csv('/lustre/groups/shared/ukbb-87065/ukbb-found/data_benedikt/splits/val_ids_seed2024.csv')['id'].values.astype(np.uint32)
test_ids  = pd.read_csv('/lustre/groups/shared/ukbb-87065/ukbb-found/data_benedikt/splits/test_ids_seed2024.csv')['id'].values.astype(np.uint32)

train_mask = np.isin(data[:,0], train_ids)
val_mask   = np.isin(data[:,0], val_ids)
test_mask  = np.isin(data[:,0], test_ids)

data[train_mask].tofile("../ukb_real_data/" + 'train.bin')
data[val_mask].tofile("../ukb_real_data/" + 'val.bin')
data[test_mask].tofile("../ukb_real_data/" + 'test.bin')

In [ ]:
#conda create -p /lustre/groups/ml01/workspace/lucas.arnoldt/delphi python=3.12 -y
#conda activate delphi
#cd /ictstr01/groups/ml01/workspace/lucas.arnoldt/projects/Delphi/
#pip install -r requirements.txt
#Change:
#torch.backends.cuda.matmul.fp32_precision = 'tf32'
#torch.backends.cudnn.conv.fp32_precision = 'tf32'
#to:
#torch.backends.cuda.matmul.allow_tf32 = True
#torch.backends.cudnn.allow_tf32 = True

#python train.py config/train_delphi.py --out_dir=delphi_trained --device=cuda 
#python evaluate_auc.py --input_path=./data/ukb_real_data --model_ckpt_path=./delphi_trained/ckpt.pt --no_event_token_rate 5 --output_path delphi_trained_auc --dataset_subset_size 5000 

#pd.read_parquet("/ictstr01/groups/ml01/workspace/lucas.arnoldt/projects/Delphi/delphi_trained_auc/df_auc_unpooled.parquet")
#pd.read_parquet("/ictstr01/groups/ml01/workspace/lucas.arnoldt/projects/Delphi/delphi_trained_auc/df_both.parquet")